In [1]:
import numpy

from cicada.additive import AdditiveProtocolSuite
from cicada.communicator import SocketCommunicator
from cicada.encoding import FixedPoint

def main(communicator):
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    c_share = protocol.add(a_share, b_share)
    c = protocol.reveal(c_share)

Note that we're using a very small field (`order=127`) and a very low-precision encoding (`precision=2`) to make the values in the logged output easier to read.

In [2]:
import logging

from cicada import transcript

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 2: --> 0 PRZS 5381736730353538238
Player 1: --> 2 PRZS 648011214005269034
Player 0: --> 1 PRZS 5281281951814190347
Player 2: <-- 1 PRZS 648011214005269034
Player 1: <-- 0 PRZS 5281281951814190347
Player 0: <-- 2 PRZS 5381736730353538238
Player 2: --> 0 GATHER 26
Player 1: --> 0 GATHER 107
Player 0: --> 0 GATHER 14
Player 2: --> 1 GATHER 26
Player 1: --> 1 GATHER 107
Player 0: <-- 2 GATHER 26
Player 2: --> 2 GATHER 26
Player 1: <-- 2 GATHER 26
Player 0: <-- 1 GATHER 107
Player 2: <-- 2 GATHER 26
Player 1: <-- 1 GATHER 107
Player 0: <-- 0 GATHER 14
Player 0: --> 1 GATHER 14
Player 0: --> 2 GATHER 14
Player 1: <-- 0 GATHER 14
Player 2: <-- 0 GATHER 14
Player 1: --> 2 GATHER 107
Player 2: <-- 1 GATHER 107


In [3]:
def main(communicator):
    transcript.log("Let's setup additive sharing!")
    protocol = AdditiveProtocolSuite(communicator, order=127, encoding=FixedPoint(precision=2))
    transcript.log("Let's share some secrets!")
    a_share = protocol.share(src=0, secret=numpy.array(2), shape=())
    b_share = protocol.share(src=1, secret=numpy.array(3), shape=())
    transcript.log("Let's add some secrets!")
    c_share = protocol.add(a_share, b_share)
    transcript.log("Let's reveal the results!")
    c = protocol.reveal(c_share)

with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler())
    SocketCommunicator.run(fn=main, world_size=3);

Player 0: Let's setup additive sharing!
Player 1: Let's setup additive sharing!
Player 2: Let's setup additive sharing!
Player 2: --> 0 PRZS 142067236959304088
Player 1: --> 2 PRZS 2182295519413834147
Player 0: --> 1 PRZS 8253388696778933758
Player 1: <-- 0 PRZS 8253388696778933758
Player 2: <-- 1 PRZS 2182295519413834147
Player 0: <-- 2 PRZS 142067236959304088
Player 1: Let's share some secrets!
Player 0: Let's share some secrets!
Player 2: Let's share some secrets!
Player 1: Let's add some secrets!
Player 2: Let's add some secrets!
Player 0: Let's add some secrets!
Player 1: Let's reveal the results!
Player 2: Let's reveal the results!
Player 0: Let's reveal the results!
Player 1: --> 0 GATHER 0
Player 2: --> 0 GATHER 102
Player 0: --> 0 GATHER 45
Player 1: --> 1 GATHER 0
Player 2: --> 1 GATHER 102
Player 0: <-- 1 GATHER 0
Player 1: <-- 1 GATHER 0
Player 0: <-- 2 GATHER 102
Player 2: --> 2 GATHER 102
Player 1: <-- 2 GATHER 102
Player 0: <-- 0 GATHER 45
Player 2: <-- 2 GATHER 102
Play

In [4]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler(fmt="# {msg}", netmsgfmt="{netmsg.verb},{netmsg.src},{netmsg.dst},{netmsg.tag},{netmsg.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Let's setup additive sharing!
# Let's setup additive sharing!
# Let's setup additive sharing!
sent,2,0,PRZS,2434541333538140002
sent,1,2,PRZS,2250661217056152343
sent,0,1,PRZS,1598044909938741730
received,1,2,PRZS,2250661217056152343
received,2,0,PRZS,2434541333538140002
received,0,1,PRZS,1598044909938741730
# Let's share some secrets!
# Let's share some secrets!
# Let's share some secrets!
# Let's add some secrets!
# Let's add some secrets!
# Let's add some secrets!
# Let's reveal the results!
# Let's reveal the results!
# Let's reveal the results!
sent,2,0,GATHER,24
sent,1,0,GATHER,117
sent,0,0,GATHER,6
sent,2,1,GATHER,24
sent,1,1,GATHER,117
received,2,0,GATHER,24
sent,2,2,GATHER,24
received,1,1,GATHER,117
received,1,0,GATHER,117
received,2,2,GATHER,24
received,2,1,GATHER,24
received,0,0,GATHER,6
sent,0,1,GATHER,6
sent,0,2,GATHER,6
received,0,1,GATHER,6
received,0,2,GATHER,6
sent,1,2,GATHER,117
received,1,2,GATHER,117


If you look carefully, you can see that this produces two nearly identical events for each message (once when the message is sent, and once when the message is received).  If you wish to eliminate the duplication, e.g. by only logging messages when they're sent, you can specify that too:

In [5]:
with transcript.record():
    transcript.set_handler(logging.getLogger(), transcript.netmsg_handler(received=False, fmt="# {msg}", netmsgfmt="{netmsg.verb},{netmsg.src},{netmsg.dst},{netmsg.tag},{netmsg.payload}"))
    SocketCommunicator.run(fn=main, world_size=3);

# Let's setup additive sharing!
# Let's setup additive sharing!
# Let's setup additive sharing!
sent,1,2,PRZS,2039392443914402498
sent,2,0,PRZS,7594320087027930088
sent,0,1,PRZS,5018401559851863016
# Let's share some secrets!
# Let's share some secrets!
# Let's share some secrets!
# Let's add some secrets!
# Let's add some secrets!
# Let's add some secrets!
# Let's reveal the results!
# Let's reveal the results!
# Let's reveal the results!
sent,2,0,GATHER,99
sent,0,0,GATHER,65
sent,1,0,GATHER,110
sent,2,1,GATHER,99
sent,1,1,GATHER,110
sent,2,2,GATHER,99
sent,0,1,GATHER,65
sent,0,2,GATHER,65
sent,1,2,GATHER,110
